# Insert CSV into MongoDB

https://www.youtube.com/watch?v=E5cSNSeBhjw&t=310s

In [76]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pymongo
import json

Initiating a database

In [77]:
dbConn = pymongo.MongoClient("mongodb://localhost:27017/")
dbname = 'us_cities'
db = dbConn[dbname]
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'us_cities')

In [78]:
print(dbConn.list_database_names())

['admin', 'config', 'crawlerDB', 'demoDB', 'local', 'test_csv_save_mongodb', 'us_cities']


Initiating a collection

In [79]:
collection_name = 'us_cities_lat_lng'
collection = db[collection_name]
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'us_cities'), 'us_cities_lat_lng')

openning the csv
cdir = os.path.dirname(__file__)
file_res = os.path.join(cdir, filepath)

In [81]:
data = pd.read_csv("C:\\Users\\140524\\Desktop\\Can_Instructure\\assignments\\weather_scrapper\\uscities.csv")
data.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500.0,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840042075
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947.0,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840019842
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441.0,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840021107
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507.0,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840019855
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591.0,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840041959


In [82]:
data['lat'] = data['lat'].round(2)
data['lng'] = data['lng'].round(2)
data['city'] = data['city'].str.replace(' ','').str.lower()
# or can data['city'] = data['city'].str.replace(' ','')apply(lambda s:s.lower())
data = data[['city','state_id','lat','lng']]
data.head()

,city,state_id,lat,lng
0,southcreek,WA,47.00,-122.39
1,roslyn,WA,47.25,-121.10
2,sprague,WA,47.30,-117.97
3,gigharbor,WA,47.34,-122.60
4,lakecassidy,WA,48.06,-122.09


data[data['zips'].values == 33065]

In [83]:
data_json = json.loads(data.to_json(orient='records'))
collection.insert_many(data_json)

Testing the collection

In [84]:
result_total = collection.find({}).limit(5)
result_total

In [85]:
for res in result_total:
    print(res)

{'_id': ObjectId('5ec28e349fa3710c565a1dd9'), 'city': 'southcreek', 'state_id': 'WA', 'lat': 47.0, 'lng': -122.39}
{'_id': ObjectId('5ec28e349fa3710c565a1dda'), 'city': 'roslyn', 'state_id': 'WA', 'lat': 47.25, 'lng': -121.1}
{'_id': ObjectId('5ec28e349fa3710c565a1ddb'), 'city': 'sprague', 'state_id': 'WA', 'lat': 47.3, 'lng': -117.97}
{'_id': ObjectId('5ec28e349fa3710c565a1ddc'), 'city': 'gigharbor', 'state_id': 'WA', 'lat': 47.34, 'lng': -122.6}
{'_id': ObjectId('5ec28e349fa3710c565a1ddd'), 'city': 'lakecassidy', 'state_id': 'WA', 'lat': 48.06, 'lng': -122.09}


In [86]:
my_db_query = {'city':'jericho','state_id':'NY'}
result = collection.find(my_db_query)
result

In [87]:
for query_result in result:
    print(query_result['lat'])
    print(query_result['lng'])

40.79
-73.54
40.79
-73.54
40.79
-73.54
40.79
-73.54


collection_to_drop = db['us_cities_lat_lng']
collection_to_drop.drop()

In [88]:
city_name = input('enter name of city:').replace(' ','').lower()
State_Id = input('enter state_id in upper case:')

enter name of city:Melville
enter state_id in upper case:NY


In [89]:
city_name, State_Id

('melville', 'NY')

In [90]:
my_db_query = {'city':city_name,'state_id':State_Id}
result = list(collection.find(my_db_query))
if not result:
    print('empty list')
else:    
    result[0]['lat']

In [91]:
for query_result in result:
        latitude = str(query_result['lat'])        
        longitude = str(query_result['lng'])

In [92]:
url = 'https://forecast.weather.gov/MapClick.php?lat='+ latitude + '&lon=' +longitude
url

'https://forecast.weather.gov/MapClick.php?lat=40.78&lon=-73.41'

In [93]:
landing_data = requests.get(url)
landing_data

<Response [200]>

In [94]:
if landing_data.status_code == requests.codes.ok:
        landing_soup = BeautifulSoup(landing_data.content, 'html.parser')
landing_soup        

<!DOCTYPE html>

<html class="no-js">
<head>
<!-- Meta -->
<meta content="width=device-width" name="viewport"/>
<link href="http://purl.org/dc/elements/1.1/" rel="schema.DC"/><title>National Weather Service</title><meta content="National Weather Service" name="DC.title"><meta content="NOAA National Weather Service National Weather Service" name="DC.description"/><meta content="US Department of Commerce, NOAA, National Weather Service" name="DC.creator"/><meta content="" name="DC.date.created" scheme="ISO8601"/><meta content="EN-US" name="DC.language" scheme="DCTERMS.RFC1766"/><meta content="weather, National Weather Service" name="DC.keywords"/><meta content="NOAA's National Weather Service" name="DC.publisher"/><meta content="National Weather Service" name="DC.contributor"/><meta content="http://www.weather.gov/disclaimer.php" name="DC.rights"/><meta content="General" name="rating"/><meta content="index,follow" name="robots"/>
<!-- Icons -->
<link href="./images/favicon.ico" rel="shor

In [95]:
week = landing_soup.findAll("div", {"class": "tombstone-container"})
week

[<div class="tombstone-container">
 <p class="period-name">Tonight<br/><br/></p>
 <p><img alt="Tonight: Partly cloudy, with a low around 51. South wind 5 to 10 mph becoming light southwest  after midnight. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Partly cloudy, with a low around 51. South wind 5 to 10 mph becoming light southwest  after midnight. "/></p><p class="short-desc">Partly Cloudy</p><p class="temp temp-low">Low: 51 °F</p></div>,
 <div class="tombstone-container">
 <p class="period-name">Friday<br/><br/></p>
 <p><img alt="Friday: A 20 percent chance of showers after noon.  Partly sunny, with a high near 70. South wind 6 to 10 mph. " class="forecast-icon" src="DualImage.php?i=bkn&amp;j=shra&amp;jp=20" title="Friday: A 20 percent chance of showers after noon.  Partly sunny, with a high near 70. South wind 6 to 10 mph. "/></p><p class="short-desc">Partly Sunny<br/>then Slight<br/>Chance<br/>Showers</p><p class="temp temp-high">High: 70 °F</p></div>,


In [96]:
Day = []
Weather = []
Temp = []
for days in week:
    temp = days.find('p', {'class': 'temp'})
    if temp:
        day = days.find('p', {'class': 'period-name'})
        Day.append(day.text)
        weather = days.find('p', {'class': 'short-desc'})
        Weather.append(weather.text)
        Temp.append(temp.text) 
    else:
        continue
   
        

In [97]:
weather_report = pd.DataFrame({'Day':Day, 'Weather': Weather, 'Temperature':Temp})
weather_report = weather_report.reset_index(drop=True)
weather_report

,Day,Weather,Temperature
0,Tonight,Partly Cloudy,Low: 51 °F
1,Friday,Partly Sunnythen SlightChanceShowers,High: 70 °F
2,FridayNight,ShowersLikely,Low: 57 °F
3,Saturday,ShowersLikely,High: 69 °F
4,SaturdayNight,ChanceShowers,Low: 52 °F
5,Sunday,Mostly Sunny,High: 67 °F
6,SundayNight,Partly Cloudy,Low: 49 °F
7,MemorialDay,Mostly Sunny,High: 68 °F
8,MondayNight,Partly Cloudy,Low: 54 °F


In [98]:
weather_report.shape[0]

9

In [100]:
from datetime import date
today = date.today()
if weather_report.shape[0] == 8:
     Dates = pd.Series(pd.date_range(today, periods = 4)).repeat([2,2,2,2]) 
elif weather_report.shape[0] == 9:
     Dates = pd.Series(pd.date_range(today, periods = 5)).repeat([2,2,2,2,1])         
for i in Dates:
    print(i)

2020-05-21 00:00:00
2020-05-21 00:00:00
2020-05-22 00:00:00
2020-05-22 00:00:00
2020-05-23 00:00:00
2020-05-23 00:00:00
2020-05-24 00:00:00
2020-05-24 00:00:00
2020-05-25 00:00:00


In [101]:
from datetime import date
today = date.today()
weather_report.index = Dates
weather_report

,Day,Weather,Temperature
2020-05-21,Tonight,Partly Cloudy,Low: 51 °F
2020-05-21,Friday,Partly Sunnythen SlightChanceShowers,High: 70 °F
2020-05-22,FridayNight,ShowersLikely,Low: 57 °F
2020-05-22,Saturday,ShowersLikely,High: 69 °F
2020-05-23,SaturdayNight,ChanceShowers,Low: 52 °F
2020-05-23,Sunday,Mostly Sunny,High: 67 °F
2020-05-24,SundayNight,Partly Cloudy,Low: 49 °F
2020-05-24,MemorialDay,Mostly Sunny,High: 68 °F
2020-05-25,MondayNight,Partly Cloudy,Low: 54 °F


In [102]:
weather_report.reset_index()
weather_report['Date'] = weather_report.index
weather_report

,Day,Weather,Temperature,Date
2020-05-21,Tonight,Partly Cloudy,Low: 51 °F,2020-05-21
2020-05-21,Friday,Partly Sunnythen SlightChanceShowers,High: 70 °F,2020-05-21
2020-05-22,FridayNight,ShowersLikely,Low: 57 °F,2020-05-22
2020-05-22,Saturday,ShowersLikely,High: 69 °F,2020-05-22
2020-05-23,SaturdayNight,ChanceShowers,Low: 52 °F,2020-05-23
2020-05-23,Sunday,Mostly Sunny,High: 67 °F,2020-05-23
2020-05-24,SundayNight,Partly Cloudy,Low: 49 °F,2020-05-24
2020-05-24,MemorialDay,Mostly Sunny,High: 68 °F,2020-05-24
2020-05-25,MondayNight,Partly Cloudy,Low: 54 °F,2020-05-25


In [72]:
#converting a dataframe into list of dictionaries
weather_list = weather_report.T.to_dict().values()
weather_list

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


dict_values([{'Day': 'Tonight', 'Weather': 'Partly Cloudy', 'Temperature': 'Low: 51 °F', 'Date': Timestamp('2020-05-21 00:00:00')}, {'Day': 'FridayNight', 'Weather': 'ShowersLikely', 'Temperature': 'Low: 57 °F', 'Date': Timestamp('2020-05-22 00:00:00')}, {'Day': 'SaturdayNight', 'Weather': 'ChanceShowers thenMostly Cloudy', 'Temperature': 'Low: 52 °F', 'Date': Timestamp('2020-05-23 00:00:00')}, {'Day': 'SundayNight', 'Weather': 'Partly Cloudy', 'Temperature': 'Low: 49 °F', 'Date': Timestamp('2020-05-24 00:00:00')}])

In [73]:
for i in weather_list:
    print(i)

{'Day': 'Tonight', 'Weather': 'Partly Cloudy', 'Temperature': 'Low: 51 °F', 'Date': Timestamp('2020-05-21 00:00:00')}
{'Day': 'FridayNight', 'Weather': 'ShowersLikely', 'Temperature': 'Low: 57 °F', 'Date': Timestamp('2020-05-22 00:00:00')}
{'Day': 'SaturdayNight', 'Weather': 'ChanceShowers thenMostly Cloudy', 'Temperature': 'Low: 52 °F', 'Date': Timestamp('2020-05-23 00:00:00')}
{'Day': 'SundayNight', 'Weather': 'Partly Cloudy', 'Temperature': 'Low: 49 °F', 'Date': Timestamp('2020-05-24 00:00:00')}


In [74]:
# print a list repeating each number twice
[i for j in zip(range(0,3),range(0,5)) for i in j]

[0, 0, 1, 1, 2, 2]

In [75]:
lst = [5,6,7,8,9,10,11,12]
lst[-10:-1]

[5, 6, 7, 8, 9, 10, 11]